In [1]:
#Import & Setup
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import DistilBertTokenizerFast
import torch


In [2]:
# Load and merge all datasets

location_relative_path = "../data/processed/cleaned/"

# Load each dataset
df = pd.read_csv(location_relative_path + "all_label.txt", sep="|", skiprows=1, names=["text", "manipulative"])
#df2 = pd.read_csv(location_relative_path + "liar_train.txt", sep="|", skiprows=1, names=["text", "manipulative"])
#df3 = pd.read_csv(location_relative_path + "mentalmanip_con.txt", sep="|", skiprows=1, names=["text", "manipulative"])
#df4 = pd.read_csv(location_relative_path + "tweets.txt", sep="|", skiprows=1, names=["text", "manipulative"])

# Merge all into one DataFrame
#df = pd.concat([df1, df2, df3, df4], ignore_index=True)

# Convert label column to integer
df['manipulative'] = df['manipulative'].astype(int)

# Preview the first few rows
print(f"Total samples: {len(df)}")
df.head()


Total samples: 3092


,text,manipulative
0,"$200,000 worth of supplies donated to PB schools",0
1,10-year-old shoot mom over VR headset,0
2,14 killed in prison clashes in Ecuador as some...,0
3,2025 The New York Times Company,0
4,23-year-old charged with murder in NH country ...,0


In [3]:
# Train/Test Split
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df['text'].tolist(), df['manipulative'].tolist(), test_size=0.2, random_state=42
)

# Print summary
print(f"Total samples: {len(df)}")
print(f"Training samples: {len(train_texts)}")
print(f"Testing samples: {len(test_texts)}")

# Preview a few training examples
print("\nSample training texts and labels:")
for i in range(3):
    print(f"Text: {train_texts[i]}")
    print(f"Manipulative: {train_labels[i]}")
    print("---")

Total samples: 3092
Training samples: 2473
Testing samples: 619

Sample training texts and labels:
Text: Bloomberg Intelligence: TikTok Algorithm to Be Secured (Podcast)
Manipulative: 0
---
Text: News outlets 'desperately concerned' for their journalists in Gaza, urge Israeli authorities to help
Manipulative: 1
---
Text: Your favorite clothing brand accused of using forced labor overseas  
Manipulative: 1
---


In [4]:
# Tokenization
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")

# Tokenize training and test texts
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

# Print summary
print(f"Training encodings keys: {list(train_encodings.keys())}")
print(f"Test encodings keys: {list(test_encodings.keys())}")

# Preview a few training samples
print("\nSample tokenized training input_ids:")
for i in range(3):
    print(f"Sample {i+1}: {train_encodings['input_ids'][i][:10]}")

print("\nSample attention masks:")
for i in range(3):
    print(f"Sample {i+1}: {train_encodings['attention_mask'][i][:10]}")


Training encodings keys: ['input_ids', 'attention_mask']
Test encodings keys: ['input_ids', 'attention_mask']

Sample tokenized training input_ids:
Sample 1: [101, 22950, 4454, 1024, 14841, 25509, 6559, 9896, 2000, 2022]
Sample 2: [101, 2739, 11730, 1005, 9652, 4986, 1005, 2005, 2037, 8845]
Sample 3: [101, 2115, 5440, 5929, 4435, 5496, 1997, 2478, 3140, 4450]

Sample attention masks:
Sample 1: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Sample 2: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Sample 3: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [5]:
#Wrap as Torch Dataset
class ManipulationDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item
    def __len__(self):
        return len(self.labels)

# Instantiate datasets
train_dataset = ManipulationDataset(train_encodings, train_labels)
test_dataset = ManipulationDataset(test_encodings, test_labels)

# Print summary
print(f"Train dataset size: {len(train_dataset)}")
print(f"Test dataset size: {len(test_dataset)}")

# Preview a few samples from train_dataset
print("\nSample training items:")
for i in range(3):
    sample = train_dataset[i]
    for key, val in sample.items():
        if val.dim() > 0:
            print(f"{key}: {val.shape} | {val[:10]}")
        else:
            print(f"{key}: {val.item()}")
    print("---")

Train dataset size: 2473
Test dataset size: 619

Sample training items:
input_ids: torch.Size([53]) | tensor([  101, 22950,  4454,  1024, 14841, 25509,  6559,  9896,  2000,  2022])
attention_mask: torch.Size([53]) | tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
labels: 0
---
input_ids: torch.Size([53]) | tensor([  101,  2739, 11730,  1005,  9652,  4986,  1005,  2005,  2037,  8845])
attention_mask: torch.Size([53]) | tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
labels: 1
---
input_ids: torch.Size([53]) | tensor([ 101, 2115, 5440, 5929, 4435, 5496, 1997, 2478, 3140, 4450])
attention_mask: torch.Size([53]) | tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
labels: 1
---


In [6]:
#(Optional) Save Datasets

tokenized_relative_path = f"../data/processed/tokenized/"

torch.save(train_dataset, tokenized_relative_path + "train_dataset.pt")
torch.save(test_dataset, tokenized_relative_path + "test_dataset.pt")


In [7]:
# Load the dataset
loaded_test_dataset = torch.load(tokenized_relative_path + "train_dataset.pt", weights_only=False)

# View a sample
sample = loaded_test_dataset[0]
for key, val in sample.items():
    if val.dim() > 0:
        print(f"{key}: {val.shape} | {val[:10]}")
    else:
        print(f"{key}: {val.item()}")  # For scalar tensors like labels


input_ids: torch.Size([53]) | tensor([  101, 22950,  4454,  1024, 14841, 25509,  6559,  9896,  2000,  2022])
attention_mask: torch.Size([53]) | tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
labels: 0
